# Step 3: Spatial Analysis
Spatial neighborhood graph, clustering, and domain detection.

In [ ]:
from pathlib import Path
import pandas as pd
import squidpy as sq

h5_path = Path('data/raw/Visium_Human_Breast_Cancer_filtered_feature_bc_matrix.h5')
spatial_root = Path('data/raw')
if not h5_path.exists() or not spatial_root.exists():
    raise FileNotFoundError('Missing raw Visium files in data/raw')

adata_spatial = sq.read.visium(str(spatial_root))
adata_spatial.obs['total_counts'] = adata_spatial.X.toarray().sum(axis=1) if hasattr(adata_spatial.X, 'toarray') else adata_spatial.X.sum(axis=1)

sq.gr.spatial_neighbors(adata_spatial)
sq.tl.spatial_clustering(adata_spatial, method='leiden', resolution=0.5)
sq.tl.spatial_autocorr(adata_spatial, mode='moran')

adata_spatial.write('data/processed/human_breast_cancer_spatial.h5ad')

spatial_stats = adata_spatial.obs['spatial_leiden'].value_counts()
pd.DataFrame({'Spatial_Cluster': spatial_stats.index, 'Spot_Count': spatial_stats.values}).to_csv('results/spatial/Human Breast Cancer_spatial_clusters.csv', index=False)

sq.pl.spatial_scatter(adata_spatial, color='spatial_leiden', save='figures/Human Breast Cancer_spatial_clusters.png', show=False)
print('Step 3 complete')